In [2]:
import os, shutil, sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

In [3]:
dataset_folder = '/Users/danielfaria/Desktop/MEI/2ºSemestre/AAII/trabalho_pratico/brain_tumor_dataset/'
dataset_dividido_folder = '/Users/danielfaria/Desktop/MEI/2ºSemestre/AAII/trabalho_pratico/brain_tumor_dataset_dividido/'
treino_folder = 'treino/'
validacao_folder = 'validacao/'
teste_folder = 'teste/'

yes = 'yes'
no = 'no'

if not os.path.isdir(dataset_dividido_folder):
  os.mkdir(dataset_dividido_folder)

if not os.path.isdir(dataset_dividido_folder + treino_folder):
  os.mkdir(dataset_dividido_folder + treino_folder)
  os.mkdir(dataset_dividido_folder + treino_folder + yes)
  os.mkdir(dataset_dividido_folder + treino_folder + no)

if not os.path.isdir(dataset_dividido_folder + validacao_folder):
  os.mkdir(dataset_dividido_folder + validacao_folder)
  os.mkdir(dataset_dividido_folder + validacao_folder + yes)
  os.mkdir(dataset_dividido_folder + validacao_folder + no)

if not os.path.isdir(dataset_dividido_folder + teste_folder):
  os.mkdir(dataset_dividido_folder + teste_folder)
  os.mkdir(dataset_dividido_folder + teste_folder + yes)
  os.mkdir(dataset_dividido_folder + teste_folder + no)

In [72]:
numb = []
for pasta in os.listdir(dataset_folder):
    if not pasta.startswith('.'):
        IMG_NUM = len(os.listdir(dataset_folder + pasta))
        for (n, FILE_NAME) in enumerate(os.listdir(dataset_folder + pasta)):
            img = dataset_folder + pasta + '/' + FILE_NAME
            if n < 0.2 * IMG_NUM:
                numb.append(n)
                shutil.copy(img, dataset_dividido_folder + teste_folder + pasta + '/' + FILE_NAME) 
            elif n < 0.7 * IMG_NUM and n not in numb:
                shutil.copy(img, dataset_dividido_folder + treino_folder + pasta + '/' + FILE_NAME) 
            else:
                shutil.copy(img, dataset_dividido_folder + validacao_folder + pasta + '/' + FILE_NAME) 

In [4]:
#Redimensionar imagens
treino_gen = ImageDataGenerator(rescale = 1./255)
teste_gen = ImageDataGenerator(rescale = 1./255)
validacao_gen = ImageDataGenerator(rescale = 1./255)

In [5]:
treino_dir = dataset_dividido_folder + treino_folder
teste_dir = dataset_dividido_folder + teste_folder
validacao_dir = dataset_dividido_folder + validacao_folder
#Aplicar a redimensão aos dados treino, teste e validação
dados_treino = treino_gen.flow_from_directory(
    treino_dir,
    target_size = (150,150),
    batch_size = 10,
    class_mode = 'binary'
    )
dados_teste = teste_gen.flow_from_directory(
    teste_dir,
    target_size = (150,150),
    batch_size = 10,
    class_mode = 'binary'
    )
dados_validacao = validacao_gen.flow_from_directory(
    validacao_dir,
    target_size = (150,150),
    batch_size = 10,
    class_mode = 'binary'
    )

Found 126 images belonging to 2 classes.
Found 52 images belonging to 2 classes.
Found 75 images belonging to 2 classes.


In [6]:
#Ver os shapes
for x, y in dados_treino:
    print("Shape dos atributos: ", x.shape)
    print("Shape das labels: ", y.shape)
    break
for x, y in dados_teste:
    print("Shape dos atributos: ", x.shape)
    print("Shape das labels: ", y.shape)
    break
for x, y in dados_validacao:
    print("Shape dos atributos: ", x.shape)
    print("Shape das labels: ", y.shape)
    break

Shape dos atributos:  (10, 150, 150, 3)
Shape das labels:  (10,)
Shape dos atributos:  (10, 150, 150, 3)
Shape das labels:  (10,)
Shape dos atributos:  (10, 150, 150, 3)
Shape das labels:  (10,)


In [7]:
#Criar o modelo com CNNs
model = Sequential()
model.add(Conv2D(60,(3,3), input_shape = (150,150,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [8]:
#Efetuar o fit ao modelo
history = model.fit(dados_teste, steps_per_epoch = 10, epochs = 10, validation_data = dados_validacao, verbose = 1)

Train for 1 steps, validate for 8 steps
1/1 [==============================] - 5s 5s/step - loss: 0.7160 - accuracy: 0.2000 - val_loss: 96.8088 - val_accuracy: 0.6133


In [9]:
#Avaliar o modelo
scores = model.evaluate_generator(dados_treino, verbose = 1)
print("Accuracy do modelo: ", scores[1] * 100)

13/13 [==============================] - 2s 162ms/step - loss: 6.1726 - accuracy: 0.6111
Accuracy do modelo:  61.11111044883728


In [3]:
#Visualizar gráficos
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

#Gráfico Accuracy
plt.plot(epochs, accuracy, 'bo', label = 'Accuracy Treino')
plt.plot(epochs, accuracy, 'b', label = 'Accuracy Teste')
plt.title('Accuracy do treino e do teste')
plt.legend()
plt.figure()

#Gráfico Loss
plt.plot(epochs, loss, 'bo', label = 'Loss Treino')
plt.plot(epochs, val_loss, 'b', label = 'Loss Teste')
plt.title('Loss do treino e do teste')
plt.legend()
plt.show()

NameError: name 'history' is not defined